## Geospatial Analysis I: global, zonal and focal operations, map algebra
Resources:

* [
GRASS GIS overview and manual](http://grass.osgeo.org/grass72/manuals/index.html)
*  [Recommendations](data_acquisition.html#commands)
and [tutorial](http://www4.ncsu.edu/~akratoc/GRASS_intro/)
how to use wxGUI from the first assignment




To run _r.mapcalc_ expressions, you can either
run the entire command in the GUI _Console_, or in case of any
problems, type or copy the expression (without the _r.mapcalc_) in the GRASS GIS Raster Map Calculator
which can be launched from _Layer Manager_ toolbar.


Text files with color rules:


* [srtmneddiff_color.txt](data/srtmneddiff_color.txt)


### Start GRASS GIS
In startup pannel set GIS Data Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For Project location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for Accessible mapset create a new mapset (called e.g. HW_map_algebra) and
click Start GRASS.

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'

Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

In [ ]:
# a proper directory is already set, download files
import urllib
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/srtmneddiff_color.txt", "srtmneddiff_color.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/srtmneddiff_color.txt", "srtmneddiff_color.txt")

Download all text files with color rules (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.

### Compute summaries

Compute areas for each category at two different resolutions.

Are results equal? Explain (see also Lecture 1).
Copy and paste the report from the output window or
save the report in a text file: _Output window_ > _Save_.
Use fixed width font (e.g., Courier, Andale Mono  in your report to preserve formatting).

In [ ]:
gs.parse_command('g.region', raster="landuse96_28m", res="12", flags='apg')
gs.read_command('r.report', map="landuse96_28m", unit="c,h,p")
gs.parse_command('g.region', raster="landuse96_28m", res="30", flags='apg')
gs.read_command('r.report', map="landuse96_28m", unit="c,h,p")

Compute areas for each category of land use for each zipcode.
Compare 27601 Raleigh with 27511 Cary.
Include only the relevant part of the table in your report.

In [ ]:
gs.read_command('r.report', map="zipcodes,landuse96_28m", unit="h,p")

Compute zonal statistics maps representing
average slope for each basin.
Add legends using _Add map elements_ in Map Display toolbar.
Reminder: _d.out.file_ means Save to graphics file for your report.

In [ ]:
gs.parse_command('g.region', raster="slope", flags='pg')
gs.run_command('r.stats.zonal', base="basin_50K", cover="slope", method="average", output="slope_avgbasin")
gs.run_command('r.colors', map="slope_avgbasin", color="gyr")
gs.run_command('d.rast', map="slope_avgbasin")
gs.run_command('d.legend', raster="slope_avgbasin", at="90,50,5,8")
gs.run_command('d.vect', map="streams", color="15:25:110")
Image(filename="map.png")

Compute zonal statistics maps representing most common land use for each basin.
When adding legend, make sure you define _List of discrete category numbers_.

In [ ]:
gs.parse_command('g.region', raster="landuse96_28m", flags='pg')
gs.run_command('r.mode', base="basin_50K", cover="landuse96_28m", output="landuse96_modebasin")
gs.run_command('d.rast', map="landuse96_modebasin")
gs.run_command('d.vect', map="streams")
gs.run_command('d.legend', raster="landuse96_modebasin", at="40,20,2,5", flags='n')
Image(filename="map.png")

### Apply neighborhood operators

Use neighborhood operator to compute land use diversity map.
How diverse is land use in NCSU area?
Before you start to display the new results,
remove all previously added map layers from the _Layer Manager_.

In [ ]:
gs.parse_command('g.region', raster="landuse96_28m", flags='pg')
gs.parse_command('r.info', map="landuse96_28m", flags='g')
gs.run_command('r.neighbors', input="landuse96_28m", output="lu_divers", method="diversity", size="7")
gs.run_command('d.erase')
gs.run_command('d.rast', map="lu_divers")
gs.run_command('d.legend', raster="lu_divers", at="70,15,5,10", flags='v')
gs.run_command('d.vect', map="streets_wake")
gs.read_command('r.report', map="lu_divers", unit="p")
Image(filename="map.png")

Use neighborhood operator to smooth the SRTM elevation map and 
compare the global statistical measures for the original and smoothed DEM.
How would size of the neighborhood influence the result?

In [ ]:
gs.parse_command('g.region', raster="elev_srtm_30m", flags='pg')
gs.run_command('r.neighbors', input="elev_srtm_30m", output="elev_srtm30m_sm5", method="average", size="5")
gs.run_command('d.rast', map="elev_srtm_30m")
gs.run_command('d.rast', map="elev_srtm30m_sm5")
gs.parse_command('r.univar', map="elev_srtm_30m", flags='g')
gs.parse_command('r.univar', map="elev_srtm30m_sm5", flags='g')
Image(filename="map.png")

### Patch multiple raster layers into a single raster

Patch raster tiles for lidar based, 6m res. DEM for Centennial Campus.

Before displaying new data, remove older map layers from _Layer Manager_.

In [ ]:
gs.parse_command('g.region', raster="el_D793_6m,el_D783_6m,el_D782_6m,el_D792_6m", flags='pg')
gs.run_command('r.patch', input="el_D793_6m,el_D783_6m,el_D782_6m,el_D792_6m", output="elevlidD_6m")
gs.run_command('r.colors', map="elevlidD_6m", raster="elevation")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elevlidD_6m")
Image(filename="map.png")

### Map Algebra
See _r.mapcalc_ manual page for syntax and functions.


If you are getting en error when running
_r.mapcalc_ in GUI _Console_ or the system
command line, launch the GUI version from _Layer Manager_
toolbar.

#### NDVI
Compute Normalized Difference Vegetation Index (NDVI).
Explain the difference between floating point and integer
handling in ndvi1, ndvi2 and ndvi3 result.

In [ ]:
gs.parse_command('g.region', raster="lsat7_2002_40", flags='pg')
gs.mapcalc("ndvi1 = (lsat7_2002_40 - lsat7_2002_30) / (lsat7_2002_40 + lsat7_2002_30)")
gs.mapcalc("ndvi2 = 1.0 * (lsat7_2002_40 - lsat7_2002_30) / (lsat7_2002_40 + lsat7_2002_30)")
gs.mapcalc("ndvi3 = float(lsat7_2002_40 - lsat7_2002_30) / float(lsat7_2002_40 + lsat7_2002_30)")
gs.parse_command('r.info', map="ndvi1", flags='rg')
gs.parse_command('r.info', map="ndvi2", flags='rg')
gs.parse_command('r.info', map="ndvi3", flags='rg')
gs.run_command('r.colors', map="ndvi3", color="ndvi")
gs.run_command('d.rast', map="ndvi3")
Image(filename="map.png")

#### Difference between DEM and DSM
Explore the difference between the SRTM DSM and lidar-based NED DEM.
Compute the map of elevation differences.

In [ ]:
gs.parse_command('g.region', raster="elev_ned_30m", flags='pg')
gs.mapcalc("srtm_ned30_dif = elev_srtm_30m - elev_ned_30m")

Create a custom color table to distinguish the negative and positive values:

In [ ]:
gs.parse_command('r.info', map="srtm_ned30_dif", flags='rg')

fp: Data range is -142.24... to 86.19...
Assign custom color table [srtmneddiff_color.txt](data/srtmneddiff_color.txt).
GUI: Right click on layer > _Properties_ > _Set color table_ > _Colors_ > _Path to rules file_.

In [ ]:
gs.run_command('r.colors', map="srtm_ned30_dif", rules="srtmneddiff_color.txt")

Zoom to computational region and switch off previous map layers. Display the difference map layer:

In [ ]:
gs.run_command('d.rast', map="srtm_ned30_dif")
gs.run_command('d.legend', raster="srtm_ned30_dif", at="70,15,5,10")
gs.run_command('d.vect', map="streets_wake")
Image(filename="map.png")
gs.parse_command('r.univar', map="elev_srtm_30m", flags='g')
gs.parse_command('r.univar', map="elev_ned_30m", flags='g')
Image(filename="map.png")

Is the srtm mostly higher or lower than elev_ned?
Which result will you use to answer the above question -
the srtm_ned30_dif map or numbers provided by _r.univar_?


#### Working with if statements
Create map of urban areas (high density and low density class)
with 0 class elsewhere.

In [ ]:
gs.parse_command('g.region', raster="landuse96_28m", flags='pg')
gs.mapcalc("urban1_30m = if(landuse96_28m == 1,1,0) + if(landuse96_28m == 2,2,0)")
gs.run_command('d.rast', map="urban1_30m")
gs.run_command('d.legend', raster="urban1_30m", at="10,30,5,8")
Image(filename="map.png")

Alternatively with logical 'or' operator and null elsewhere:

In [ ]:
gs.mapcalc("urban2_30m = if(landuse96_28m == 1 || landuse96_28m == 2,landuse96_28m,null())")
gs.run_command('d.rast', map="urban2_30m")
Image(filename="map.png")

#### Handling null values





Create mask for low lying developed areas (e.g. vulnerable to flooding)
with elevation between 60 and 100m and land use 1 or 2.
Set the region, display the input maps and create a MASK.
Before you start new computations, remove or switch off previous map layers
in the _Layer Manger_.
You may also zoom to computational region in _Map Display_
once you set a new one.

In [ ]:
gs.parse_command('g.region', raster="elevation", flags='pg')
gs.run_command('d.erase')
gs.run_command('d.rast', map="elevation")
gs.run_command('d.rast', map="landuse96_28m")
gs.mapcalc("low_elev_developed = if((elevation < 100 && elevation > 60) && (landuse96_28m == 1 || landuse96_28m == 2),1,null())")
gs.run_command('r.mask', raster="low_elev_developed")
Image(filename="map.png")

Command _r.mask_ creates a raster map "MASK" in your mapset.


Remove "low_elev_developed" layer if it was added.
Display watersheds to see the mask effect:

In [ ]:
gs.run_command('d.rast', map="basin_50K")
Image(filename="map.png")

Disable mask, and display basin_50K again
to show that the mask was removed.

In [ ]:
gs.run_command('r.mask', flags='r')
gs.run_command('d.rast', map="basin_50K")
Image(filename="map.png")

#### Using coordinates of moving window in map algebra

Replace section of SRTM DSM with NED DEM elevation.
Try to explain how this _r.mapcalc_ expression works.

In [ ]:
gs.mapcalc("elev_combined = if(y() < 224274. && x() > 637455., elevation, elev_srtm_30m)")

### Optional -  various additional useful tasks
#### Tilted plane
Create a raster map representing tilted plane (e.g., geologic fault):

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.mapcalc("tiltplane = 0.2*(0.1*row()+col())+50")
gs.mapcalc("tiltpl_under = if(tiltplane < elev_lid792_1m + 2,tiltplane,null())")

View the elevation surface and subsurface plane in 3D.
_Switch off all layers in the layer manager except for elev_lid792_1m and tiltpl_under_.
Change display to 3D view, adjust viewing position to a view from South.
Save an image for your report.


#### Map subsets
Use map algebra to create map subsets.
Change region to the airphoto tile 792 and resolution 10m.
Since we will work in different are, it is a good idea to remove
all previously used map layers from the _Layer Manager_.

In [ ]:
gs.parse_command('g.region', raster="ortho_2001_t792_1m", res="10", flags='apg')
gs.run_command('d.erase')
gs.run_command('d.rast', map="ortho_2001_t792_1m")
Image(filename="map.png")

Create a subset of the map elevation for this subregion.

In [ ]:
gs.mapcalc("elevation_792_10m = elevation")
gs.run_command('d.rast', map="elevation_792_10m")
Image(filename="map.png")

Zoom out to see that it is a subset.

#### Work with NULL and MASK

Set the mask and check its effect.

In [ ]:
gs.run_command('d.rast', map="elevation")
gs.run_command('d.vect', map="streets_wake")
gs.run_command('r.mask', raster="urban", maskcats="55")
gs.run_command('d.rast', map="elevation")
Image(filename="map.png")

Remove mask:

In [ ]:
gs.run_command('r.mask', flags='r')

#### Zonal statistics
Compute % area for each category in each zipcode.
Which zipcode has the most high density development?

In [ ]:
gs.read_command('r.stats', input="zipcodes,landuse96_28m", flags='pl')

#### Working with relative coordinates
Enter the expression on a single line without \
Again, it is a good idea to remove the previously used map layers
before we start to work on a new task.

In [ ]:
gs.parse_command('g.region', raster="elev_srtm_30m", flags='pg')
gs.run_command('d.erase')
gs.mapcalc("elev_srtm30m_smooth = (elev_srtm_30m[-1,-1]   \
           + elev_srtm_30m[-1,0] + elev_srtm_30m[-1,1] \
           + elev_srtm_30m[0,-1] + elev_srtm_30m[0,0]  \
           + elev_srtm_30m[0,1]  + elev_srtm_30m[1,-1] \
           + elev_srtm_30m[1,0]  + elev_srtm_30m[1,1])/9.")
Image(filename="map.png")

Assign the resulting map the same color table as the original.
Compare global statistics

In [ ]:
gs.run_command('r.colors', map="elev_srtm30m_smooth", raster="elev_srtm_30m")
gs.parse_command('r.univar', map="elev_srtm_30m", flags='g')
gs.parse_command('r.univar', map="elev_srtm30m_smooth", flags='g')
gs.run_command('d.rast', map="elev_srtm_30m")
gs.run_command('d.rast', map="elev_srtm30m_smooth")
Image(filename="map.png")

In [ ]:
# end the GRASS session
os.remove(rcfile)